# Alzheimer's Prediccion (Multi-Class) 
* Ver [Alzheimer's Prediccion](https://github.com/jesymca/Alzheimers-Prediccion) para un resumen de este proyecto

### Descripciones de variables
* `directory.id` Id asignado a los directorios de imágenes. No de datos LONi
* `Subject (PTID)` ID del participante
* `RID` ID de la lista de participantes
* `Image.Data.ID` ID de resonancia magnética
* `Modality` Tipo de imagen
* `Visit` 1=exploración de detección
* `Acq.Date` Fecha de resonancia magnética
* `DX.bl` Diagnóstico al inicio
* `EXAMDATE` Fecha del examen
* `AGE` Edad al inicio
* `PTGENDER` Sexo
* `PTEDUCAT` Años de Educación
* `PTETHCAT` Etnia
* `PTRACCAT` Carrera
* `APOE4` Genotipo APOE4
* `MMSE` Puntuación MMSE en la prueba cognitiva
* `Imputed_genotype` Designación específica del desafío, VERDADERO = tiene genotipos imputados
* `APOE Genotype` Genotipo APOE alelo 1 y alelo 2 genotipos
* `Dx Codes for Submission` El LMCI en los datos ADNI es equivalente a MCI en la prueba. Esta columna solo convierte LMCI->MCI

# Nota: este cuaderno está siendo refactorizado
* Más AED
* División adecuada de prueba
* Use **AUC** como métrica principal para la evaluación, así como **Precisión**

In [1]:
import numpy as np
import sklearn as sk
import pandas as pd
import os

In [2]:
#Leer Datos
dat = pd.read_csv('AD_Challenge_Training_Data_Clinical_Updated_7.22.2014/ADNI_Training_Q3_APOE_CollectionADNI1Complete 1Yr 1.5T_July22.2014.csv')

#Chequear NA values, 1 NA por imputed_genotype
# dat.isnull().sum()

FileNotFoundError: [Errno 2] No such file or directory: 'AD_Challenge_Training_Data_Clinical_Updated_7.22.2014/ADNI_Training_Q3_APOE_CollectionADNI1Complete 1Yr 1.5T_July22.2014.csv'

### Remove NA

In [ ]:
dat = dat.dropna()
dat.isnull().sum().sum()

In [ ]:
#Echar un vistazo a los datos
dat.head(3)

### Remover ID's y otras características

In [ ]:
X = dat
Y = dat['DX.bl']
del dat

#Remover columnas inecesarias (features), Se eliminan primeras 9 columnas y 'Dx codes for submission'
remove_columns = list(X.columns)[0:9]
remove_columns.append('Dx Codes for Submission')
print('Removing columns:', remove_columns)

X = X.drop(remove_columns, axis=1)

features = list(X.columns)
X.head(5)

## Explorando Data Analysis (EDA)

In [ ]:
numerical_vars = ['AGE', 'MMSE', 'PTEDUCAT']
cat_vars = list(set(features) - set(numerical_vars))

print('Categorical variable distributions:\n')

for var in cat_vars:
    print('\nDistribution of', var)
    
    print(X[var].value_counts())

#### Notas de distribución de Var categórico
* El origen étnico y la raza son variables separadas
* La distribución de hispanos, negros, blancos y asiáticos no es exactamente igual a la distribución étnica/racial general estadounidense
* El genotipo APOE con 3 en el primer lugar parece ser una abrumadora mayoría `3,3` y `3,4`
* Más hombres en estudio que mujeres

### Distribuciones de variables numéricas
* Las 3 distribuciones parecen un poco sesgadas a la derecha
* **Edad**: `media = 75`, `min=55`, `max=89` - parece que no hay datos malos obvios
* **MMSE**: `mean = 26.9`, `min=20`, `max=30` - parece que no hay datos erróneos obvios, NOTA: MMSE $\in ~(0,30)$
* **PTEDUCAT**: `mean = 15.6`, `min=6`, `max=20` - parece que no hay datos erróneos obvios. 13-14 años es probablemente escuela secundaria/GED y 17-18 años es probable que tenga una licenciatura

In [ ]:
from matplotlib import pyplot as plt
%matplotlib inline

print('Numerical Var Distributions:\n')

for var in numerical_vars:
    plt.hist(X[var], bins=10)
    plt.title(var + ' Distribution')
    plt.show()
    
    # descriptive stats
    print(X[var].describe())

### Analizar la distribución objetivo (y)
* Alto número (304) de deterioro limitado (LMCI). ¿Quizás la vejez? O los pacientes que se evalúan no son pacientes de edad avanzada al azar, sino pacientes que ya se sabe que tienen problemas cognitivos.

In [ ]:
plt.bar(Y.value_counts().index, Y.value_counts())
plt.show()

## Pre-processing

### Convertir variables categóricas a 1-Hot
* La próxima vez use `sklearn.preprocessing.OneHotEncoder`

In [ ]:
#para cada var categórico, convertir a codificación 1-hot
for var in cat_vars:
    print('Converting', var, 'to 1-hot encoding')
    
    #obtenga 1-hot y reemplace la columna original con >= 2 categorías como columnas
    one_hot_df = pd.get_dummies(X[var])
    X = pd.concat([X, one_hot_df], axis=1)
    X = X.drop(var, axis=1)
    
X.head(4)